# Tweets about eating disorders
## 04. Training BERT Models and saving predictions to csv
 
## Training and Evaluation of BERT models

Through this notebook I keep the predictions of the BERT models applied to the 4 categories.

In [22]:
import os
import pandas as pd
import numpy as np
import json
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
tweets = pd.read_csv('tweets_cleaned.csv', encoding='utf8', error_bad_lines=False)

C:\Users\MICROSOFT\Anaconda3\envs\ED\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
tweets.columns=['num1','stream_group','text_orig','f1_commercial','f2_politics','f3_ed','f4_family','f5_edpatient','f6_proed','f7_offensive','f8_info','f9_scientific','f10_sad','hashtag','text','segmented','text_similar']

In [24]:
cols=['f1_commercial','f2_politics','f3_ed','f4_family','f5_edpatient','f6_proed','f7_offensive','f8_info','f9_scientific','f10_sad']

In [25]:
tweets1 = tweets.copy().drop(['f1_commercial', 'f2_politics', 'f3_ed', 'f4_family', 'f7_offensive', 'f10_sad'], axis=1)

In [26]:
tweets1.head(2)

,num1,stream_group,text_orig,f5_edpatient,f6_proed,f8_info,f9_scientific,hashtag,text,segmented,text_similar
0,0,1,"RT @beatED: Learn more about anorexia and bulimia, as well as other eating disorders, here: https://t.co/Aj2HbjRH39 @BBCPanorama #BBCPanorama",0,0,1,0,['BBCPanorama'],"['learn', 'anorexia', 'bulimia', 'well', 'eating', 'disorder']",bbc panorama,[]
1,1,1,"A woman tries to balance her relationships with her mother and teenage daughter while under the shadow of #anorexia in the atmospheric British drama #BodyofWater. \n\nAt Depot from Friday, book now: https://lewesdepot.org/film/body-of-water\n",0,0,0,0,"['anorexia', 'BodyofWater']","['woman', 'try', 'balance', 'relationship', 'mother', 'teenage', 'daughter', 'shadow', 'atmospheric', 'british', 'drama', 'depot', 'friday', 'book']",anorexia bodyof water,[]


In [27]:
import pandas as pd
import numpy as np

import spacy
import nltk
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import regex as re
import string
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from simpletransformers.classification import ClassificationModel


import io

In [28]:
punctuations = "¡!#$%&'()*+,-./:;<=>¿?@[\]^_`{|}~"

def read_txt(filename):
    list = []
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.readlines()
        for line in data:
            list.append(str(line).replace('\n', ''))
    return list

stopwords = read_txt('english_stopwords.txt')

stemmer = SnowballStemmer('english')


def clean_accents(tweet):
    tweet = re.sub(r"[àáâãäå]", "a", tweet)
    tweet = re.sub(r"ç", "c", tweet)
    tweet = re.sub(r"[èéêë]", "e", tweet)
    tweet = re.sub(r"[ìíîï]", "i", tweet)
    tweet = re.sub(r"[òóôõö]", "o", tweet)
    tweet = re.sub(r"[ùúûü]", "u", tweet)
    tweet = re.sub(r"[ýÿ]", "y", tweet)

    return tweet

def clean_tweet(tweet, stem = False):
    tweet = tweet.lower().strip()
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    tweet = re.sub(r'http?:\/\/\S+', '', tweet)
    tweet = re.sub(r'www?:\/\/\S+', '', tweet)
    tweet = re.sub(r'\s([@#][\w_-]+)', "", tweet)
    tweet = re.sub(r"\n", " ", tweet)
    tweet = clean_accents(tweet)
    tweet = re.sub(r"\b(a*ha+h[ha]*|o?l+o+l+[ol]*|x+d+[x*d*]*|a*ja+[j+a+]+)\b", "<risas>", tweet)
    for symbol in punctuations:
        tweet = tweet.replace(symbol, "")
    tokens = []
    for token in tweet.strip().split():
        if token not in punctuations and token not in stopwords:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [29]:
tweets1['text_cleaned'] = tweets['text_orig'].apply(lambda s : clean_tweet(s))
print(tweets1['text_cleaned'].head(5))

0                                                                   rt learn anorexia bulimia well eating disorders
1      woman tries balance relationships mother teenage daughter shadow atmospheric british drama depot friday book
2                           full diagnosis like therapist legit told mom anorexia nevrususu poor mom sat like 👁👄👁oh
3                                          highercalorie diets patients anorexia nervosa shorten hospital stays via
4    tw ed ment freddie pissed amazing way got anorexic bulimic actress play foodobsessed character never sit right
Name: text_cleaned, dtype: object


In [30]:
tweets1.head(2)

,num1,stream_group,text_orig,f5_edpatient,f6_proed,f8_info,f9_scientific,hashtag,text,segmented,text_similar,text_cleaned
0,0,1,"RT @beatED: Learn more about anorexia and bulimia, as well as other eating disorders, here: https://t.co/Aj2HbjRH39 @BBCPanorama #BBCPanorama",0,0,1,0,['BBCPanorama'],"['learn', 'anorexia', 'bulimia', 'well', 'eating', 'disorder']",bbc panorama,[],rt learn anorexia bulimia well eating disorders
1,1,1,"A woman tries to balance her relationships with her mother and teenage daughter while under the shadow of #anorexia in the atmospheric British drama #BodyofWater. \n\nAt Depot from Friday, book now: https://lewesdepot.org/film/body-of-water\n",0,0,0,0,"['anorexia', 'BodyofWater']","['woman', 'try', 'balance', 'relationship', 'mother', 'teenage', 'daughter', 'shadow', 'atmospheric', 'british', 'drama', 'depot', 'friday', 'book']",anorexia bodyof water,[],woman tries balance relationships mother teenage daughter shadow atmospheric british drama depot friday book


In [31]:
df = tweets1.copy()
X = df['text_cleaned']
Y1 = df['f5_edpatient']
Y2 = df['f6_proed']
Y3 = df['f8_info']
Y4 = df['f9_scientific']

In [32]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.3, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, Y2, test_size=0.3, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X, Y3, test_size=0.3, random_state=42)
X4_train, X4_test, y4_train, y4_test = train_test_split(X, Y4, test_size=0.3, random_state=42)

In [33]:
y1_test.value_counts(normalize=True)

1    0.51773
0    0.48227
Name: f5_edpatient, dtype: float64

In [34]:
train_args ={"reprocess_input_data": True,
           "fp16":False,
             "evaluate_during_training": False,
             "evaluate_during_training_verbose":False,
             "learning_rate":2e-5,
             "train_batch_size":32,
             "eval_batch_size":32,
           "num_train_epochs": 15, 'overwrite_output_dir': True, "evaluation_strategy":'epochs'
            }

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

def calcule_f1(df):
    return(df['tp'] / (df['tp'] + 0.5 * (df['fp'] + df['fn'])))

import torch
import gc
from tqdm import tqdm


dfEval1 = pd.DataFrame()

In [35]:
indexBERT = ['BERT','RoBERTa','DistilBERT','CamemBERT','Albert','FlauBERT','RobBERT']

train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

train_df2 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df2 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df3 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df3 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df4 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df4 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })


In [36]:
N_ITER = 1

In [16]:
gc.collect()
torch.cuda.empty_cache()

In [37]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''


for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    predictions1, raw_outputs1 = model1.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1
    test_df1_c.to_csv('predictions/cate1-bert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-bert-total-wrong_preds.csv', encoding='utf-8')
    del model1
    gc.collect()
    torch.cuda.empty_cache()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.6437082637134355, 'tp': 232, 'tn': 231, 'fp': 41, 'fn': 60, 'auroc': 0.8794319097502015, 'auprc': 0.867996224095909, 'f1': 0.8209219858156028, 'acc': 0.8209219858156028, 'eval_loss': 1.1224142329560385}


  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [17]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''


for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    predictions1, raw_outputs1 = model1.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1
    test_df1_c.to_csv('predictions/cate1-bert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-bert-total-wrong_preds.csv', encoding='utf-8')
    del model1
    gc.collect()
    torch.cuda.empty_cache()


#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)
    gc.collect()
    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    predictions1c, raw_outputs1c = model1c.predict(test_df1['text_cleaned'].tolist())
    del model1c
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1c
    test_df1_c.to_csv('predictions/cate1-roberta-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-roberta-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)
    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    predictions1d, raw_outputs1d = model1d.predict(test_df1['text_cleaned'].tolist())
    del model1d
    gc.collect()
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1d
    test_df1_c.to_csv('predictions/cate1-distilbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-distilbert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    gc.collect()
    predictions1e, raw_outputs1e = model1e.predict(test_df1['text_cleaned'].tolist())
    del model1e
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1e
    test_df1_c.to_csv('predictions/cate1-camembert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-camembert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)
    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    predictions1f, raw_outputs1f = model1f.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1f
    test_df1_c.to_csv('predictions/cate1-albert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-albert-total-wrong_preds.csv', encoding='utf-8')
    del model1f
    gc.collect()



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)
    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    predictions1g, raw_outputs1g = model1g.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1g
    test_df1_c.to_csv('predictions/cate1-flaubert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-flaubert-total-wrong_preds.csv', encoding='utf-8')

    del model1g
    gc.collect()
    





Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.6348096850826357, 'tp': 237, 'tn': 224, 'fp': 48, 'fn': 55, 'auroc': 0.8918588839645447, 'auprc': 0.8916579994703276, 'f1': 0.8173758865248227, 'acc': 0.8173758865248227, 'eval_loss': 1.0308471206161711}


  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

KeyError: 'robbert'

In [18]:
for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)
    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    predictions1h, raw_outputs1h = model1h.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1h
    test_df1_c.to_csv('predictions/cate1-robbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate1-robbert-total-wrong_preds.csv', encoding='utf-8')

    del model1h
    gc.collect()

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.bias', 'clas

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.585604119956847, 'tp': 228, 'tn': 219, 'fp': 53, 'fn': 64, 'auroc': 0.8733380338436744, 'auprc': 0.8550747701169905, 'f1': 0.7925531914893617, 'acc': 0.7925531914893617, 'eval_loss': 1.0798336995972528}


  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [19]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''


for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    predictions1, raw_outputs1 = model1.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1
    test_df1_c.to_csv('predictions/cate2-bert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-bert-total-wrong_preds.csv', encoding='utf-8')
    del model1
    gc.collect()
    torch.cuda.empty_cache()


#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)
    gc.collect()
    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    predictions1c, raw_outputs1c = model1c.predict(test_df1['text_cleaned'].tolist())
    del model1c
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1c
    test_df1_c.to_csv('predictions/cate2-roberta-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-roberta-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)
    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    predictions1d, raw_outputs1d = model1d.predict(test_df1['text_cleaned'].tolist())
    del model1d
    gc.collect()
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1d
    test_df1_c.to_csv('predictions/cate2-distilbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-distilbert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    gc.collect()
    predictions1e, raw_outputs1e = model1e.predict(test_df1['text_cleaned'].tolist())
    del model1e
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1e
    test_df1_c.to_csv('predictions/cate2-camembert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-camembert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)
    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    predictions1f, raw_outputs1f = model1f.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1f
    test_df1_c.to_csv('predictions/cate2-albert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-albert-total-wrong_preds.csv', encoding='utf-8')
    del model1f
    gc.collect()



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)
    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    predictions1g, raw_outputs1g = model1g.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1g
    test_df1_c.to_csv('predictions/cate2-flaubert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-flaubert-total-wrong_preds.csv', encoding='utf-8')

    del model1g
    gc.collect()
    

for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)
    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    predictions1h, raw_outputs1h = model1h.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1h
    test_df1_c.to_csv('predictions/cate2-robbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate2-robbert-total-wrong_preds.csv', encoding='utf-8')

    del model1h
    gc.collect()



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.6135581943810172, 'tp': 92, 'tn': 394, 'fp': 33, 'fn': 45, 'auroc': 0.9146481136429683, 'auprc': 0.7885041069299649, 'f1': 0.8617021276595744, 'acc': 0.8617021276595744, 'eval_loss': 0.8373330194089148}


  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [20]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })
'''

train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })
'''
train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })
'''


for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    predictions1, raw_outputs1 = model1.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1
    test_df1_c.to_csv('predictions/cate3-bert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-bert-total-wrong_preds.csv', encoding='utf-8')
    del model1
    gc.collect()
    torch.cuda.empty_cache()


#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)
    gc.collect()
    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    predictions1c, raw_outputs1c = model1c.predict(test_df1['text_cleaned'].tolist())
    del model1c
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1c
    test_df1_c.to_csv('predictions/cate3-roberta-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-roberta-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)
    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    predictions1d, raw_outputs1d = model1d.predict(test_df1['text_cleaned'].tolist())
    del model1d
    gc.collect()
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1d
    test_df1_c.to_csv('predictions/cate3-distilbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-distilbert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)

    gc.collect()
    predictions1e, raw_outputs1e = model1e.predict(test_df1['text_cleaned'].tolist())
    del model1e
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1e
    test_df1_c.to_csv('predictions/cate3-camembert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-camembert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)
    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    predictions1f, raw_outputs1f = model1f.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1f
    test_df1_c.to_csv('predictions/cate3-albert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-albert-total-wrong_preds.csv', encoding='utf-8')
    del model1f
    gc.collect()



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)
    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    predictions1g, raw_outputs1g = model1g.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1g
    test_df1_c.to_csv('predictions/cate3-flaubert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-flaubert-total-wrong_preds.csv', encoding='utf-8')

    del model1g
    gc.collect()
    

for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)
    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    predictions1h, raw_outputs1h = model1h.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1h
    test_df1_c.to_csv('predictions/cate3-robbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-robbert-total-wrong_preds.csv', encoding='utf-8')

    del model1h
    gc.collect()



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.5722900802065563, 'tp': 145, 'tn': 307, 'fp': 71, 'fn': 41, 'auroc': 0.8489930022188086, 'auprc': 0.6993011658971136, 'f1': 0.8014184397163121, 'acc': 0.8014184397163121, 'eval_loss': 0.9721511337492201}


  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [21]:
'''
torch.load(model1, 'model1.pt')
model1.train_model(train_df1)
result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
print(result1)
del model1
gc.collect()
'''
#gc.collect()
#torch.cuda.empty_cache()
limitsave=0

'''
train_df1 = pd.DataFrame({ 'text_cleaned': X1_train, 'target': y1_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X1_test, 'target': y1_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X2_train, 'target': y2_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X2_test, 'target': y2_test })

train_df1 = pd.DataFrame({ 'text_cleaned': X3_train, 'target': y3_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X3_test, 'target': y3_test })
'''

train_df1 = pd.DataFrame({ 'text_cleaned': X4_train, 'target': y4_train })
test_df1 = pd.DataFrame({ 'text_cleaned': X4_test, 'target': y4_test })



for i in range(0,N_ITER):
    model1 = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
        use_cuda = True,
        args=train_args
    )
    model1.train_model(train_df1)
    result1, model_outputs1, wrong_predictions1 = model1.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1)
    predictions1, raw_outputs1 = model1.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1
    test_df1_c.to_csv('predictions/cate4-bert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate4-bert-total-wrong_preds.csv', encoding='utf-8')
    del model1
    gc.collect()
    torch.cuda.empty_cache()


#dfResultsModelsTotal = pd.read_csv('dfResultsModelsTotal.csv', encoding='utf8', error_bad_lines=False)
        
for i in range(0,N_ITER):
    model1c = ClassificationModel(
    "roberta", "roberta-base",
        use_cuda = True,
        args=train_args
    )
    model1c.train_model(train_df1)
    gc.collect()
    result1c, model_outputs1c, wrong_predictions1c = model1c.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1c)
    predictions1c, raw_outputs1c = model1c.predict(test_df1['text_cleaned'].tolist())
    del model1c
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1c
    test_df1_c.to_csv('predictions/cate4-roberta-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate4-roberta-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1d = ClassificationModel(
    "distilbert", "distilbert-base-cased",
        use_cuda = True,
        args=train_args
    )
    model1d.train_model(train_df1)
    result1d, model_outputs1d, wrong_predictions1d = model1d.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1d)
    predictions1d, raw_outputs1d = model1d.predict(test_df1['text_cleaned'].tolist())
    del model1d
    gc.collect()
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1d
    test_df1_c.to_csv('predictions/cate4-distilbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate4-distilbert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1e = ClassificationModel(
    "camembert", "camembert-base",
        use_cuda = True,
        args=train_args
    )
    model1e.train_model(train_df1)

    result1e, model_outputs1e, wrong_predictions1e = model1e.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1e)
    gc.collect()
    predictions1e, raw_outputs1e = model1e.predict(test_df1['text_cleaned'].tolist())
    del model1e
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1e
    test_df1_c.to_csv('predictions/cate3-camembert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate3-camembert-total-wrong_preds.csv', encoding='utf-8')


for i in range(0,N_ITER):
    model1f = ClassificationModel(
    "albert", "albert-base-v1",
        use_cuda = True,
        args=train_args
    )
    model1f.train_model(train_df1)
    result1f, model_outputs1f, wrong_predictions1f = model1f.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1f)
    predictions1f, raw_outputs1f = model1f.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1f
    test_df1_c.to_csv('predictions/cate4-albert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate4-albert-total-wrong_preds.csv', encoding='utf-8')
    del model1f
    gc.collect()



for i in range(0,N_ITER):
    model1g = ClassificationModel(
    "flaubert","flaubert/flaubert_base_cased",
        use_cuda = True,
        args=train_args
    )
    model1g.train_model(train_df1)
    result1g, model_outputs1g, wrong_predictions1g = model1g.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1g)
    predictions1g, raw_outputs1g = model1g.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1g
    test_df1_c.to_csv('predictions/cate4-flaubert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate4-flaubert-total-wrong_preds.csv', encoding='utf-8')

    del model1g
    gc.collect()
    

for i in range(0,N_ITER):
    model1h = ClassificationModel(
    "roberta","pdelobelle/robbert-v2-dutch-base",
        use_cuda = True,
        args=train_args
    )
    model1h.train_model(train_df1)
    result1h, model_outputs1h, wrong_predictions1h = model1h.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)
    print(result1h)
    predictions1h, raw_outputs1h = model1h.predict(test_df1['text_cleaned'].tolist())
    test_df1_c = test_df1.copy()
    test_df1_c['predicted'] = predictions1h
    test_df1_c.to_csv('predictions/cate4-robbert-total-preds.csv', encoding='utf-8')
    dft = test_df1_c.copy()
    dft[dft['target'] != dft['predicted']].to_csv('predictions/cate4-robbert-total-wrong_preds.csv', encoding='utf-8')

    del model1h
    gc.collect()



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

  0%|          | 0/1313 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/564 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

{'mcc': 0.7758964242830648, 'tp': 101, 'tn': 420, 'fp': 23, 'fn': 20, 'auroc': 0.9549652071712404, 'auprc': 0.8600297674181437, 'f1': 0.9237588652482269, 'acc': 0.9237588652482269, 'eval_loss': 0.4026339472168022}


  0%|          | 0/564 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
import sys,gc,torch
#sys.getsizeof(wrong_predictions4g)
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import gc
del variables
gc.collect()
print(torch.cuda.memory_summary(device=0, abbreviated=False))
